In [1]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords
 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder,LabelBinarizer,OneHotEncoder
from sklearn.cross_validation import train_test_split

from sklearn.pipeline import Pipeline
 
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,hamming_loss

/home/jayap/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_csv("../../data/processed/train.csv")
test_df = pd.read_csv("../../data/processed/test.csv")

In [3]:
le  = LabelBinarizer()
cat_arr = le.fit_transform(train_df['cat'])

cat_df = pd.DataFrame(data=cat_arr,columns=le.classes_)

train_df[pd.isnull(train_df['subcat'])]  = 'NaN'
subcat_arr = le.fit_transform(train_df['subcat'])

subcat_df = pd.DataFrame(data=subcat_arr,columns=le.classes_)

Y = pd.concat([cat_df,subcat_df],axis=1)
Y.drop('NaN',inplace=True,axis=1)
print('The labels are',Y)

('The labels are',       Billing  Clinic/Doctors Office  Employment  Hospitalization  \
0           0                      1           0                0   
1           0                      1           0                0   
2           0                      0           0                1   
3           0                      1           0                0   
4           0                      0           1                0   
5           0                      0           0                1   
6           0                      0           0                1   
7           0                      0           0                1   
8           0                      0           0                1   
9           0                      1           0                0   
10          0                      1           0                0   
11          0                      1           0                0   
12          0                      1           0                0   
13          0  

In [4]:
col_todrop = ['cat','subcat','Unnamed: 0','Unnamed: 0.1','document_id','no_pages','no_words','ques_cnt']
for col in col_todrop:
    train_df.drop(col,axis=1,inplace=True)  
    
col_todrop = ['Unnamed: 0','Unnamed: 0.1','document_id','no_pages','no_words','ques_cnt']
for col in col_todrop:
    test_df.drop(col,axis=1,inplace=True)

X_train, X_test, Y_train, Y_test = train_test_split(train_df['content'], Y, test_size=0.33, random_state=42)

KeyError: "labels ['Unnamed: 0.1'] not contained in axis"

In [ ]:
train_df.subcat.unique()

In [ ]:
pipe = Pipeline([
    ('cv', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('BR_GB', BinaryRelevance(GaussianNB())),
])

In [43]:
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
#classifier = BinaryRelevance(GaussianNB())

# train
pipe.fit(X_train, Y_train)

# predict
prediction = pipe.predict(X_test)

print('Test accuracy is {}'.format(accuracy_score(Y_test, prediction)))

print('Hamming Test accuracy is {}'.format(hamming_loss(Y_test, prediction)))

Hamming Test accuracy is 0.117988394584
